# Clustering

To do:
- filter raccoons and get 0-15
- apply K-means with silhouette index
- use weight as the metric
- sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html


In [1]:
import glob

import numpy as np

import networkx as nx

f_name = '../../raccoon_data/weighted_raccoon_matrix_46.graphml'

In [ ]:
dir_name = '../../raccoon_data/weighted_raccoon_matrix_*.graphml'
all_files = glob.glob(dir_name)

In [2]:
# Filtering: remove edges > 15
g = nx.read_graphml(f_name)
remove = [node for node in g.nodes if int(node) > 15]
g.remove_nodes_from(remove)

In [3]:
# interaction time
dist = np.array([g.edges[key]['weight'] for key in g.edges.keys() ])

In [4]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [54]:
# Just check interaction values
print( np.array([ g.edges[key]['weight'] for key in g.edges.keys() ]))

[   115    916    271     34    599     56   1563      4   6520    899
    624    589      6    207    520     24     37      6     42 155841]


In [ ]:
## To do: 1/(1+log(interaction))
A = nx.to_numpy_matrix(g)

In [46]:
A

matrix([[0.00000e+00, 0.00000e+00, 1.15000e+02, 9.16000e+02, 0.00000e+00,
         2.71000e+02, 3.40000e+01, 5.99000e+02, 5.60000e+01, 0.00000e+00,
         0.00000e+00],
        [0.00000e+00, 0.00000e+00, 1.56300e+03, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 6.52000e+03, 4.00000e+00,
         8.99000e+02],
        [1.15000e+02, 1.56300e+03, 0.00000e+00, 0.00000e+00, 6.24000e+02,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 5.89000e+02, 6.00000e+00,
         2.07000e+02],
        [9.16000e+02, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 0.00000e+00, 5.20000e+02, 0.00000e+00, 0.00000e+00,
         2.40000e+01],
        [0.00000e+00, 0.00000e+00, 6.24000e+02, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
         0.00000e+00],
        [2.71000e+02, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
         0.00000e+00, 0.00000e+00, 3.70000e+01, 0.00000e+00, 0.00000e+0

In [55]:
X = 1/(1+np.log(A))

<ipython-input-55-a48e41db9660>:1: RuntimeWarning: divide by zero encountered in log
  X = 1/(1+np.log(A))


In [64]:
kmeans_model = KMeans(n_clusters=3, random_state=1, precompute_distances=True).fit(X)
labels = kmeans_model.labels_

In [65]:
# Check if a big value has the effect expected!!!
silhouette_score(X, labels, metric="precomputed")

-0.2065319661114245

In [ ]:
# The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [66]:
kmeans_model = KMeans(n_clusters=5, random_state=1).fit(X)
labels = kmeans_model.labels_

In [67]:
silhouette_score(X, labels, metric="precomputed")

-0.45454545454545453

In [ ]:
prev = silhouette_score(X, labels, metric="precomputed")
clusters = 3
while True:
    if (cur > prev):
        clusters += 1
        kmeans_model = KMeans(n_clusters=clusters, random_state=1, precompute_distances=True).fit(X)
        labels = kmeans_model.labels_

        prev = cur
        cur = silhouette_score(X, labels, metric="precomputed")
        
    if 